In [80]:
import random
from multiprocessing import Pool

import numpy as np
from tqdm.notebook import tqdm, trange


In [81]:
N = 24  # число субъединиц

In [82]:
K = np.zeros((N + 1, N + 1, N + 1, N + 1))
for i in range(N + 1):
    for j in range(N + 1):
        for k in range(N + 1):
            for l in range(N + 1):
                a = i + j
                b = k + l
                # if a == 1 and b == 1:
                #   K[i,j,k,l]=1
                # if a == 2 and b == 2:
                #   K[i,j,k,l]=1
                # if (a == 4 and b == 2) or (a == 2 and b == 4):
                #   K[i,j,k,l]=1
                if a == 6 and b == 6:
                    K[i, j, k, l] = 1
                if a == 12 and b == 12:
                    K[i, j, k, l] = 1

indx = np.argwhere(K > 0.01)

In [ ]:
A = np.zeros((N+1,N+1,N+1))
for i in range(N+1):
    for j in range(N+1):
        a = i+j
        if a< N+1:
            A[i,j,k] = 1

In [83]:
def model(_C, delta_time):  # уравнения Смолуховского
    dC = np.zeros((N + 1, N + 1))
    dC = np.einsum('ij,kl,ijkl,ija,klb->ab',_C,_C,K,A,A)/2 - np.einsum('ij,ijkl->kl',K,C)
    # for [i, j, k, l] in indx:
    #     dC[i + k, j + l] += 1 / 2 * _C[i, j] * _C[k, l] * K[i, j, k, l]
    #     dC[i, j] -= K[i, j, k, l] * _C[i, j] * _C[k, l]
    return _C + dC * delta_time

In [84]:
def get_b():
    b = np.zeros(25)
    b[0] = 0.015
    b[1] = 0.025
    b[2] = 0.038
    b[3] = 0.05
    b[4] = 0.061
    b[5] = 0.068
    b[6] = 0.068
    b[7] = 0.065
    b[8] = 0.061
    b[9] = 0.059
    b[10] = 0.059
    b[11] = 0.061
    b[12] = 0.055
    b[13] = 0.052
    b[14] = 0.047
    b[15] = 0.043
    b[16] = 0.036
    b[17] = 0.03
    b[18] = 0.025
    b[19] = 0.02
    b[20] = 0.017
    b[21] = 0.015
    b[22] = 0.011
    b[23] = 0.009
    b[24] = 0.007
    b = [0.0040782,  0.01298694, 0.02030641, 0.02340803, 0.02375074, 0.03095877,
 0.04973027 ,0.06252962, 0.06053318, 0.0524339,  0.05017999, 0.05999055,
 0.06524802, 0.05391921, 0.03848055, 0.02961201, 0.02858626, 0.02762937,
 0.01945947, 0.01024822, 0.00584425, 0.00461172, 0.00398311, 0.00234085,
 0.00058389]
    b = np.array(b)
    b /= np.sum(b)
    return b

In [85]:
peaks = [0.8404011571463119,
         0.7945252162031439,
         0.7565027674486093,
         0.7240338993470374,
         0.6957011425876662,
         0.670568808802902,
         0.6479866902539299,
         0.6274845888270858,
         0.6087114326964933,
         0.59139816643289,
         0.5753340223941958,
         0.5603508228778717,
         0.546312224280532,
         0.5331061496493349,
         0.5206393684159613,
         0.5088334710519952,
         0.497621899879081,
         0.4869476823216445,
         0.4767616474002418,
         0.46702109557436516,
         0.4576886778234583,
         0.44873157363966143,
         0.4401207537122128,
         0.431830436904474,
         0.4238376219588875]


def gaussian(x, amplitude, mean, stddev):
    return np.sqrt(1 / (2 * np.pi)) * amplitude * np.exp(-(x - mean) ** 2 / (2 * stddev ** 2))


def smooth(peaks, num, intensity):
    if intensity == 0:
        return np.zeros(25)
    Fer_sigma = 0.011
    FerSumo_sigma = 0.024
    sigma = FerSumo_sigma * (24 - num) / 24.0 + Fer_sigma * num / 24.0
    peak_center = peaks[num]
    smoothed_peak = np.zeros(25)
    for i in range(1, 24):
        smoothed_peak[i] = gaussian(peaks[i], intensity, peak_center, sigma) * (peaks[i - 1] - peaks[i + 1]) / 2
    smoothed_peak[0] = gaussian(peaks[0], intensity, peak_center, sigma) * (peaks[0] - peaks[1])
    smoothed_peak[24] = gaussian(peaks[24], intensity, peak_center, sigma) * (peaks[23] - peaks[24])
    return smoothed_peak / sum(smoothed_peak) * intensity


def smooth_profile(p_24):
    result = np.zeros(25)
    for i in range(0, 25):
        result += smooth(peaks, i, p_24[i])
    return result

In [86]:
E = np.identity(N + 1)


def back_prop(c, b):
    dC = np.zeros((N + 1, N + 1))
    data = np.zeros(N + 1)
    for j in range(N + 1):
        data[j] = c[-1][j, N - j]
    # data = smooth_profile(data)
    s = np.sum(data)
    data /= s
    d = np.zeros(N + 1)
    for i in range(N + 1):
        d[i] = 2 * (data[i] - b[i])
    d = d / s - np.sum(data * d) / (s * s)
    for i in range(N + 1):
        dC[i, N - 1] = d[i]

    for i in range(len(c) - 1, -1, -1):
        dPrev = np.zeros((N + 1, N + 1, N + 1, N + 1))
        _C = c[i]
        for [i, j, k, l] in indx:
            dPrev[i + k, j + l, i, j] += (1 - E[i, k] * E[j, l]) * K[i, j, k, l] * _C[k, l] / 2
            dPrev[i + k, j + l, k, l] += (1 - E[i, k] * E[j, l]) * K[i, j, k, l] * _C[i, j] / 2
            dPrev[i + k, j + l, i, j] += E[i, k] * E[j, l] * K[i, j, k, l] * _C[k, l]

            dPrev[i, j, i, j] -= 2 * K[i, j, i, j] * _C[i, j] * E[i, k] * E[j, l]
            dPrev[i, j, k, l] -= K[i, j, k, l] * _C[i, j] * (1 - E[i, k] * E[j, l])

        dPrev *= STEP
        dPrev += np.einsum('ij,kl->iklj', E, E)
        dC = np.einsum('kl,klij->ij', dC, dPrev)

        # print(dC[6,0])
    return dC

In [87]:
# np.einsum('ij,kl->ijkl',E,E)

In [88]:
def solver(C0, TIME, STEP):
    prev = C0
    c = [C0]
    # times = [-STEP]
    for t in np.arange(0, TIME, STEP):
        C_new = model(prev, STEP)
        prev = C_new
        c.append(prev)
    return prev, back_prop(c, get_b())

In [89]:
def count(state):
    dc = None

    state = np.array(state)
    state /= sum(state)
    a = 0.1

    for i in range(100):
        if dc is not None:
            state[0] -= dc[0, 6] * a
            state[1] -= dc[1, 5] * a
            state[2] -= dc[2, 4] * a
            state[3] -= dc[3, 3] * a
            state[4] -= dc[4, 2] * a
            state[5] -= dc[5, 1] * a
            state[6] -= dc[6, 0] * a
            # a = a * 0.99
            for i in range(7):
                state[i] = max(0, min(1, state[i]))
        c = np.zeros((N + 1, N + 1))
        c[0, 6] = state[0]
        c[1, 5] = state[1]
        c[2, 4] = state[2]
        c[3, 3] = state[3]
        c[4, 2] = state[4]
        c[5, 1] = state[5]
        c[6, 0] = state[6]

        q, dc = solver(c, T, STEP)
        data = np.zeros(N + 1)
        for j in range(N + 1):
            data[j] = q[j, N - j]
        print('delta:', np.sum(((data - get_b()) ** 2)))
        print('a:', a)
        print('state:', state)
    return state

Solver params

In [90]:
STEP = 0.1
T = 100

Fitting params

In [91]:
iter_number = 30
min_delta = 1000000
min_data = None
batch_size = 16
best = 5
multy = 2

others

In [92]:
import multiprocessing

NUM_THREADS = multiprocessing.cpu_count()
b = get_b()
res = {}

In [93]:
NUM_THREADS

8

In [94]:
# C = np.zeros((N+1,N+1))
# C[0,6] = 0.81773414
# C[1,5] = 0.6510148
# C[2,4] = 0.24049913
# C[3,3] = 0.18639066
# C[4,2] = 0.09990727
# C[5,1] = 0.50415137
# C[6,0] = 0.50301096
# print(solver(C, 100,0.1)[0])

In [95]:
with Pool(1) as p:
    for i in trange(iter_number):
        if len(res) == 0:
            batch = np.random.random((batch_size, 7))
        else:
            batch = np.random.random((batch_size, 7))
            for num, k in enumerate(res):
                for j in range(multy):
                    for l in range(7):
                        batch[multy * num + j, l] = min(max(res[k][l] + random.random() / 20 - 1 / 40, 0), 1)
        # tmp = list(tqdm(p.imap(count, batch), total=len(batch)))
        tmp = count(state=batch[0])
        print(tmp)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

delta: 0.03818791426976647
a: 0.1
state: [0.09092333 0.22465367 0.13436913 0.0070337  0.10053422 0.22894645
 0.21353951]


  0%|          | 0/1000 [00:00<?, ?it/s]

...

delta: 0.12152450352216707
a: 0.1
state: [0. 0. 0. 0. 0. 1. 1.]


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

Process ForkPoolWorker-4:


KeyboardInterrupt: 